In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [3]:
df = pd.read_csv('data/서울 월세.csv')
df.head()

,구분,질문,답변
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요
2,월세,NaN,"안녕하세요. 답변드려요.​네, 서울 청년월세지원은 신청자에게 결과를 카톡으로 연락해..."
3,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...
4,월세,서울월세지원 10번 받아서 끝났는데 또 신청해서 또 받을수 있나요,답변드립니다최대 10개월 지원받고 200만원까지 가능한데 생애 1회만 가능합니다그러...


In [4]:
df.isna()

,구분,질문,답변
0,False,False,False
1,False,False,False
2,False,True,False
3,False,False,False
4,False,False,False
...,...,...,...
345,False,False,False
346,False,False,False
347,False,False,False
348,False,False,False


In [5]:
df = df.dropna()
df

,구분,질문,답변
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요
3,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...
4,월세,서울월세지원 10번 받아서 끝났는데 또 신청해서 또 받을수 있나요,답변드립니다최대 10개월 지원받고 200만원까지 가능한데 생애 1회만 가능합니다그러...
5,월세,"서울시에서 하는 서울 월세지원에 선정되었는데4.5,6,7,8월 금액이 9월 말에 d...",괜찮습니다
...,...,...,...
345,월세,이번에 하는 서울청년월세지원 쉐어하우스도 해당되나요?,"안녕하세요지식인 대출분야(담보대출,전세자금대출,신용대출,개인회생) 상담전문 뱅크솔루..."
346,월세,서울에서 자취중입니다. 1.5억 전세 원룸에 살고있는데 전세가 그대로 목적물변경으...,어 음...월세가 너무 부담 되지 않을까요?여건이 되고 지금 집이 괜찮다면 그냥 그...
347,월세,당장 오늘 집보러 가는데 아는게 없네요... 보통 월세 보증금이 5천 이상 하더라구요,서울 월세 집을 얻을때 알아둬야 할게 있을까요당장 오늘 집보러 가는데 아는게 없네요...
348,월세,16일부터 신청가능한 서울 청년월세지원 관련해서 메일이 왔는데 자격 조건이 월세 6...,안녕하세요. 서울생활 행복도우미 120다산콜센터 지식파트너입니다. ◈ 2020년 ...


In [6]:
df.shape

(257, 3)

- 정규화

In [7]:
# \u200b가 출력되는 문제 해결
df['답변'] = df['답변'].str.replace('\u200b', ' ')
df

,구분,질문,답변
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요
3,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...
4,월세,서울월세지원 10번 받아서 끝났는데 또 신청해서 또 받을수 있나요,답변드립니다최대 10개월 지원받고 200만원까지 가능한데 생애 1회만 가능합니다그러...
5,월세,"서울시에서 하는 서울 월세지원에 선정되었는데4.5,6,7,8월 금액이 9월 말에 d...",괜찮습니다
...,...,...,...
345,월세,이번에 하는 서울청년월세지원 쉐어하우스도 해당되나요?,"안녕하세요지식인 대출분야(담보대출,전세자금대출,신용대출,개인회생) 상담전문 뱅크솔루..."
346,월세,서울에서 자취중입니다. 1.5억 전세 원룸에 살고있는데 전세가 그대로 목적물변경으...,어 음...월세가 너무 부담 되지 않을까요?여건이 되고 지금 집이 괜찮다면 그냥 그...
347,월세,당장 오늘 집보러 가는데 아는게 없네요... 보통 월세 보증금이 5천 이상 하더라구요,서울 월세 집을 얻을때 알아둬야 할게 있을까요당장 오늘 집보러 가는데 아는게 없네요...
348,월세,16일부터 신청가능한 서울 청년월세지원 관련해서 메일이 왔는데 자격 조건이 월세 6...,안녕하세요. 서울생활 행복도우미 120다산콜센터 지식파트너입니다. ◈ 2020년 ...


- 인코딩 후 데이터 셋 저장

In [8]:
model.encode(df.질문[0])

array([-6.37543559e-01, -5.96005201e-01,  5.26120603e-01, -1.09748840e+00,
        4.27429974e-01, -2.28559271e-01,  6.99419618e-01,  2.25589946e-01,
       -1.23890951e-01, -3.75541389e-01,  1.38638645e-01,  1.44819319e-02,
       -3.26404095e-01, -5.07465526e-02, -3.83976176e-02,  2.84743682e-03,
        6.99049473e-01,  1.33706564e-02,  6.61491632e-01, -5.21394253e-01,
        2.81567246e-01, -2.24805057e-01, -4.38134998e-01,  3.93550038e-01,
        1.18402764e-01,  3.93500179e-01, -3.04593354e-01, -1.24491833e-01,
       -1.13956802e-01,  1.81591868e-01,  2.79118478e-01,  2.31944263e-01,
       -2.83285528e-01, -3.29954997e-02,  9.17070433e-02, -3.68109494e-02,
       -4.07587737e-02, -5.08818746e-01, -5.25256395e-01, -4.46302183e-02,
       -2.80666590e-01,  1.67830899e-01,  2.90538549e-01,  1.79496169e-01,
        4.33418125e-01, -6.05979860e-01,  3.61493289e-01,  5.11142790e-01,
       -5.39832175e-01, -1.58559874e-01, -1.77395508e-01, -3.49945724e-01,
       -1.48209751e-01,  

In [9]:
df['embedding'] = pd.Series([[]] * len(df))     #  더미데이터
df.shape

(257, 4)

In [10]:
df.embedding = df.질문.map(lambda x: list(model.encode(x)))
df.head(3)

,구분,질문,답변,embedding
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,...","[-0.63754356, -0.5960052, 0.5261206, -1.097488..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요,"[-0.580972, -0.23177795, 0.4248164, -0.6200277..."
3,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...,"[0.18744174, 0.18414854, 0.24230862, -0.493406..."


In [11]:
df.to_csv('data/서울월세_dataset.csv', index=False)

- 챗봇

In [12]:
import json
wdf = pd.read_csv('data/서울월세_dataset.csv')
wdf.embedding = wdf.embedding.apply(json.loads)
wdf.head(3)

,구분,질문,답변,embedding
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,...","[-0.63754356, -0.5960052, 0.5261206, -1.097488..."
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요,"[-0.580972, -0.23177795, 0.4248164, -0.6200277..."
2,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...,"[0.18744174, 0.18414854, 0.24230862, -0.493406..."


In [13]:
text = '서울 월세 싼곳 추천해주세요'
embedding = model.encode(text)

In [14]:
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
wdf.head(3)

,구분,질문,답변,embedding,유사도
0,월세,제 상황에 어떤 선택이 좋은지 모르겠어서 질문글 남깁니다.집 컨디션은 비슷한 것 같...,"월세가 차라리 더 좋아 보입니다.전체적으로 좀 싸게 들어가면 전세가 좋기는 하지만,...","[-0.63754356, -0.5960052, 0.5261206, -1.097488...",0.529952
1,월세,서울 주거 포털에서 청년월세지원을 신청했는데 아직도 심사중이라고 뜹니다 월세지원 못...,결과 발표 11월중 예정이라고 돼 있어요,"[-0.580972, -0.23177795, 0.4248164, -0.6200277...",0.551198
2,월세,인천 연수구 송도국제도로 123 오네스타 랑 최대한 가까운 곳으로 추천 부탁드립니다...,1.신림동.독산동.신월동.시흥동.가산동 등등 입니다그러나 보통 원룸 월세가 40~4...,"[0.18744174, 0.18414854, 0.24230862, -0.493406...",0.766916


In [15]:
wdf.유사도.idxmax()

6

In [16]:
wdf.iloc[wdf.유사도.idxmax()]

구분                                                          월세
질문           서울에 월세 싼곳이 많은 이유가 몬가여? 부동산 앱으로 확인하니깐 60/60 이거나...
답변           안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 최홍준 변호...
embedding    [0.03520562, -0.17468536, 0.30488896, -0.76517...
유사도                                                    0.78079
Name: 6, dtype: object

In [17]:
wdf.답변[wdf.유사도.idxmax()]

'안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 최홍준 변호사입니다.  부동산 시세의 경우 수요와 공급으로 정해집니다.  직접 눈으로 확인한것은 아니지만 60/60의 경우 사진상의 크기보다 매우 작은 방이기 때문에 금애이 낮은 것 일수도 있습니다.  수요와 공급으로 금액은 조정될 수밖에 없습니다.'

In [21]:
text = '서울 월세 싼곳'
embedding = model.encode(text)
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
answer = wdf.답변[wdf.유사도.idxmax()]
answer

'안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 최홍준 변호사입니다.  부동산 시세의 경우 수요와 공급으로 정해집니다.  직접 눈으로 확인한것은 아니지만 60/60의 경우 사진상의 크기보다 매우 작은 방이기 때문에 금애이 낮은 것 일수도 있습니다.  수요와 공급으로 금액은 조정될 수밖에 없습니다.'